In [ ]:
import pandas as pd

df = pd.read_csv('../../data/intermediate/初步清洗_比赛数据_2.csv')

core_metrics = [
    '课堂参与',
    '课后复习',
    '延伸阅读',
    '课前预学',
    '专业课解决问题能力'
]

# 如果年级是中文，先转换为数字以便排序
grade_mapping_reverse = {'大一': 1, '大二': 2, '大三': 3, '大四': 4, '大五': 5}
df['年级数字'] = df['年级'].map(grade_mapping_reverse)

# 按数字年级分组，计算每个核心指标的平均值
agg_dict = {metric: 'mean' for metric in core_metrics}

df_grouped_by_grade = df.groupby('年级数字').agg(agg_dict).reset_index()
df_grouped_by_grade = df_grouped_by_grade.sort_values('年级数字').reset_index(drop=True)

# 添加回中文年级名称
grade_mapping = {1: '大一', 2: '大二', 3: '大三', 4: '大四', 5: '大五'}
df_grouped_by_grade['年级名称'] = df_grouped_by_grade['年级数字'].map(grade_mapping)

# 计算每个年级所有核心指标的平均值（学业成熟度综合得分）
df_grouped_by_grade['学业成熟度综合得分'] = df_grouped_by_grade[core_metrics].mean(axis=1).round(2)

# 查看聚合后的结果
print("按年级聚合后的平均分:")
print(df_grouped_by_grade[['年级数字', '年级名称'] + core_metrics + ['学业成熟度综合得分']].round(2))

# 重命名列以便后续使用
df_grouped_by_grade.rename(columns={'年级数字': '年级'}, inplace=True)

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import pandas as pd
import numpy as np

# 设置中文字体
plt.rcParams['font.sans-serif'] = ['SimHei']  # 用来正常显示中文标签
plt.rcParams['axes.unicode_minus'] = False    # 用来正常显示负号

# 创建图表
plt.figure(figsize=(12, 8))

# 定义颜色和标记样式
colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd']
markers = ['o', 's', '^', 'D', 'v']

# 绘制每条折线
for i, metric in enumerate(core_metrics):
    plt.plot(df_grouped_by_grade['年级名称'],
             df_grouped_by_grade[metric],
             marker=markers[i],
             linewidth=2,
             markersize=6,
             color=colors[i],
             label=metric)

# 添加数据标签（可选，如果数据点不多可以显示）
for metric in core_metrics:
    for j, grade in enumerate(df_grouped_by_grade['年级名称']):
        score = df_grouped_by_grade[metric].iloc[j]
        plt.annotate(f'{score:.1f}',
                    (grade, score),
                    textcoords="offset points",
                    xytext=(0,5),
                    ha='center',
                    fontsize=8)

# 设置图表标题和标签
plt.title('各年级核心指标得分趋势对比', fontsize=16, fontweight='bold')
plt.xlabel('年级', fontsize=12)
plt.ylabel('得分', fontsize=12)

# 添加图例
plt.legend(loc='best', fontsize=10)

# 设置网格
plt.grid(True, linestyle='--', alpha=0.7)

# 设置y轴范围，确保所有数据点都能清晰显示
y_min = df_grouped_by_grade[core_metrics].min().min() * 0.9
y_max = df_grouped_by_grade[core_metrics].max().max() * 1.1
plt.ylim(y_min, y_max)

# 调整布局
plt.tight_layout()

# 显示图表
plt.show()

# 也可以选择创建子图分别显示每个指标（备选方案）
fig, axes = plt.subplots(2, 3, figsize=(15, 10))
axes = axes.flatten()

for i, metric in enumerate(core_metrics):
    ax = axes[i]
    ax.plot(df_grouped_by_grade['年级名称'],
            df_grouped_by_grade[metric],
            marker='o',
            linewidth=2,
            markersize=6,
            color=colors[i])

    # 添加数据标签
    for j, score in enumerate(df_grouped_by_grade[metric]):
        ax.annotate(f'{score:.1f}',
                   (df_grouped_by_grade['年级名称'].iloc[j], score),
                   textcoords="offset points",
                   xytext=(0,5),
                   ha='center')

    ax.set_title(f'{metric}趋势', fontsize=12)
    ax.set_xlabel('年级')
    ax.set_ylabel('得分')
    ax.grid(True, linestyle='--', alpha=0.7)

# 隐藏多余的子图
if len(core_metrics) < len(axes):
    for i in range(len(core_metrics), len(axes)):
        axes[i].set_visible(False)

plt.suptitle('各核心指标分年级趋势分析', fontsize=16, fontweight='bold')
plt.tight_layout()
plt.show()